In [1]:
import rpy2
import scipy
import logging
import warnings
import anndata2ri
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sb
import decoupler as dc
import scrublet as scr
import decoupler as dc
from scipy import sparse
from anndata import AnnData
from tabnanny import verbose
import matplotlib.pyplot as plt
from gsva_prep import prep_gsva
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from typing import Optional, Union
from matplotlib.pyplot import rcParams
from functions import pathway_analyses
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from rpy2.robjects.conversion import localconverter

In [2]:
def get_sys_dpi(width, height, diag):
    '''
    obtain dpi of system
    
    w: width in pixels (if unsure, go vist `whatismyscreenresolution.net`)
    h: height in pixels
    d: diagonal in inches
    '''
    w_inches = (diag**2/ (1 + height**2/width**2))**0.5
    return round(width/w_inches)

In [3]:
# # Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# # Automatically convert rpy2 outputs to pandas dataframes
# pandas2ri.activate()
# anndata2ri.activate()
# %load_ext rpy2.ipython

warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

rcParams['figure.dpi'] = get_sys_dpi(1512, 982, 14.125)
#rcParams['figure.figsize']=(4,4) #rescale figures

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()



-----
anndata     0.8.0
scanpy      1.9.3
-----
PIL                         9.5.0
absl                        NA
anndata2ri                  1.1
appnope                     0.1.3
asttokens                   NA
attr                        23.1.0
backcall                    0.2.0
cffi                        1.15.1
comm                        0.1.3
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.6.7
decorator                   5.1.1
decoupler                   1.4.0
deprecate                   0.3.2
dot_parser                  NA
executing                   1.2.0
fsspec                      2023.6.0
functions                   NA
google                      NA
gsva_prep                   NA
h5py                        3.9.0
igraph                      0.10.4
ipykernel                   6.23.2
ipywidgets                  8.0.6
jedi                        0.18.2
jinja2                      3.1.2
joblib        

# **hdWGCNA Pseudotime**

In [4]:
%%R
suppressPackageStartupMessages({
    library(WGCNA)
    library(Matrix)
    library(viridis)
    library(harmony)
    library(ggpubr)
    library(tictoc)
    library(RColorBrewer)
    library(Hmisc)
    library(corrplot)
    library(grid)
    library(gridExtra)
    library(igraph)
    library(ggrepel)
    library(conflicted)
    library(readxl)


    # single-cell analysis package
    library(Seurat)
    library(monocle3)
    library(SeuratWrappers)

    # plotting and data science packages
    library(tidyverse)
    library(cowplot)
    library(patchwork)

    # co-expression network analysis packages:
    library(WGCNA)
    library(hdWGCNA)

    # gene enrichment packages
    library(enrichR)
    library(GeneOverlap)


    library(GSEABase)
    library(GSVA) 
# needs to be run every time you start R and want to use %>%
})

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

# optionally enable multithreading
# enableWGCNAThreads(nThreads = 4)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In addition: Warning messages:
1: package ‘IRanges’ was built under R version 4.3.1 
2: package ‘GenomeInfoDb’ was built under R version 4.3.1 
3: package ‘GSVA’ was built under R version 4.3.1 


## **Prepare data**

Now, we load the preprocessed and annotated data for downstream analysis.

In [5]:
save_prefix = 'allen_mtg'
map_meta = True
deg_method =  'DESeq2-Wald'
test_names = ['late_vs_early', 'early_vs_no', 'late_vs_no', 'ad_vs_no']
filter_genes = "TRUE"
subject_ids_for_study = {'allen_mtg': 'individualID',
                        'leng_sfg': 'PatientID',
                        'leng_etc': 'PatientID'}

subject_id = subject_ids_for_study[save_prefix]     # for leng this is `PatientID` for mathys is 'Subject', and allen is 'individualID'
gene_celltype_threshold = 0.10          # determines number of cells the gene must be expressed in 
covariates = ['None']                   # list of covariates to be accounted for in regression.
gene_selection = 'custom'               # specifies the gene selection method when setting up seurat object for WGCNA. The th
celltypes = ["OPC"]  #["Excitatory", "Inhibitory", "Astrocyte", "Microglia", "Oligodendrocyte", "OPC"]
metadata = f'../data/raw/{save_prefix}/{save_prefix}_metadata.csv' 
meta = pd.read_csv(metadata, encoding_errors='ignore')

gene_selection = 'custom'   # specifies the gene selection method when setting up seurat object for WGCNA. The th
                            # Posible values are "custom", "fraction", "variable"
                            # If custom, a list of genes must be passed.

gene_set_select = 'diff_exp'      # If gene_selection = 'custom'. This specifies how to obtain the list of
                                  # genes to pass into `SetupForWGCNA`. # The posible values are "diff_exp", "overlap", "all"

## **Load Seurat object for Network Visualizations**

In [6]:
%%R -i subject_id -i gene_celltype_threshold -i celltypes -i test_names -i save_prefix -i gene_selection -i gene_set_select -o seurat_obj

seurat_obj <- list()

for (cell_type in celltypes) {
    
    print(paste0('Loading data for hdWGCNA Experiment in ', toupper(cell_type)))
    tryCatch({
        seurat_obj[[cell_type]] <- readRDS(paste0("../results/hdWGCNA/SeuratObject/", save_prefix, '/', cell_type, '_hdWGCNA_object.rds'))
        print(seurat_obj[[cell_type]])
    
    }, error = function(e){
        NULL
    }, message = function(m){
        print(paste0('Could not load data for ', toupper(cell_type)))
    })

}

print('loaded data')

[1] "Loading data for hdWGCNA Experiment in OPC"


An object of class Seurat 
65852 features across 2838 samples within 1 assay 
Active assay: originalexp (65852 features, 2000 variable features)
 3 dimensional reductions calculated: pca, harmony, umap
[1] "loaded data"


In [7]:
# convert nested list of Seurat object into Rpy2 object 
seurat_obj =  robjects.ListVector(
                        {
                            cell_type: seurat_obj[cell_type]
                     
                            for cell_type in seurat_obj.keys()
                        }
                    )

# **Monocle3 pseudotime analysis**

Here we perform trajectory analysis of the data. 

Although we've identified several modules associated with the pathological alterations, it is unclear if these pathways play a role in the progression, rather than just a single state, of the disease. This way, we can capture the pathological progression that is  insufficiently captured in singlular cell states. 

Current single cell lineage tracing algorithms now allow us to take a single tissue sample (with cells presumably at varying stages of disease progression/degradation within the single sample) and recreate a “pseudo-time” scale whereby we infer which cells are early on in the disease process versus those much closer to a final degenerative step based on the evolution of their gene expression profiles. 

Here, we apply one such method to the different cells from normal and AD cohorts to better elucidate genes potentially relevant to progression of AD.

Since the data contains different disease stages, we can investigate the pathological trajectories. This analysis is centred around the concept of 'pseudotime'. In pseudotime analysis a latent variable is inferred based on which the cells are ordered. This latent variable is supposed to measure the differentiation state along a trajectory.

Pseudotime analysis is complicated when there are multiple trajectories in the data. In this case, the trajectory structure in the data must first be found before pseudotime can be inferred along each trajectory. The analysis is then called 'trajectory inference'.

Once the pseudotime variable is inferred, we can test for modules that vary continuously along pseudotime. These modules are seen as being associated with the trajectory, and may play a regulatory role in the potential pathological trajectory that the analysis found.

Here, we measure the trajectory from `no-pathology` group to `late-pathology` group in a cell-type-specific manner, just as described in the [**Morabito. et. al. 2021**](https://www.nature.com/articles/s41588-021-00894-z). We also investigate which modules vary along pseudotime.

Based on a recent comparison of pseudotime methods [**Saelens et al., 2018**](https://doi.org/10.1038/s41587-019-0071-9), we have selected the top performing 'Monocle3'.

- `'Monocle3'` performed best for complex tree structures and was used in the [**Morabito. et. al. 2021**](https://www.nature.com/articles/s41588-021-00894-z) paper.

As the complexity of trajectories are generally not known, it is adviseable to compare trajectory inference outputs.



In [8]:
%%R -i seurat_obj -o seurat_obj -o cds

fig_dir = paste0('../results/hdWGCNA/Pseudotime/Graph/', save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

cds <- list()
for (cell_type in names(seurat_obj)){

  print(paste0('Running UMAP on ', toupper(cell_type)))

  seurat_obj[[cell_type]] <- RunUMAP(seurat_obj[[cell_type]], 
                                      reduction='harmony',
                                      n.neighbors=15,  
                                      dims=1:30,
                                      min.dist=0.1)

  print(paste0(toupper(cell_type), ' pseudotime analysis with monocle3'))

  # convert the seurat object to CDS
  cds[[cell_type]] <- as.cell_data_set(seurat_obj[[cell_type]])

  # run the monocle clustering
  cds[[cell_type]] <- cluster_cells(cds=cds[[cell_type]], reduction_method='UMAP', 
                                    cluster_method = c("louvain"))

  # learn graph for pseudotime
  cds[[cell_type]] <- learn_graph(cds[[cell_type]])

  # plot the pseudotime graph:
  p1 <- plot_cells(
    cds = cds[[cell_type]],
    color_cells_by = "pathology.group",
    show_trajectory_graph = TRUE,
    label_principal_points = TRUE
  ) 

  # plot the UMAP partitions from the clustering algorithm
  p2 <- plot_cells(
    cds = cds[[cell_type]],
    color_cells_by = "partition",
    show_trajectory_graph = FALSE
  )

  pdf(paste0(fig_dir, cell_type, '_umap_monocle.pdf'),  width=8, height=4)
  print(p1 + p2)
  dev.off()
}

[1] "Running UMAP on OPC"
[1] "OPC pseudotime analysis with monocle3"
  |======================================================================| 100%


To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session
15:10:49 UMAP embedding parameters a = 1.577 b = 0.8951
15:10:49 Read 2838 rows and found 30 numeric columns
15:10:49 Using Annoy for neighbor search, n_neighbors = 15
15:10:49 Building Annoy index with metric = cosine, n_trees = 50
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
15:10:49 Writing NN index file to temp file /var/folders/70/42xwxjh94x5f164djlw1hwpc0000gv/T//RtmpShB2ET/file2d8f145f07ad
15:10:49 Searching Annoy index using 1 thread, search_k = 1500
15:10:49 Annoy recall = 100%
15:10:50 Commencing smooth kNN distance calibration using 1 thread with target n_neighbors = 15
15:10:52 Initializing from normalized Laplacian + noise (using irlba)
15:10:52 Commencing optimization for 500 epochs, with 65970 positive edges
Using method '

In [9]:
# convert nested list of Seurat object into Rpy2 object 
seurat_obj =  robjects.ListVector(
                        {
                            cell_type: seurat_obj[cell_type]
                     
                            for cell_type in seurat_obj.keys()
                        }
                    )

cds =  robjects.ListVector(
                        {
                            cell_type: cds[cell_type]
                     
                            for cell_type in cds.keys()
                        }
                    )

## **Learn Graph**

The `Monocle3` `learn_graph` function builds a principal graph in a dimensionally reduced space of the dataset. 

Here we can see the principal graph overlaid on the UMAP. The principal graph will serve as the basis of the pseudotime trajectories. 

Next, we have to select the starting node, or principal node, of the graph as the origin of the pseudotime. 

We choose the principal node by manually inspecting the pseudotime graph for all celltypes pdf file. We choose a node that largely overlaps with cells in the -`no-pathology`

In [10]:
principal_node = {}

for cell_type in celltypes:

    principal_node[cell_type] = input(f'please choose principal node for {cell_type}: ').split(',')
    principal_node[cell_type] = [node.capitalize() for node in principal_node[cell_type]]

principal_node =  robjects.ListVector(
                        {
                            cell_type: robjects.vectors.StrVector(principal_node[cell_type])
                     
                            for cell_type in celltypes
                        }
                    )

In [11]:
%R -i principal_node 

print(principal_node)

$OPC
[1] "Y_239"




In [12]:
%%R -i seurat_obj -o seurat_obj -i cds

fig_dir = paste0('../results/hdWGCNA/Pseudotime/Trajectory/', save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}


for (cell_type in names(seurat_obj)){

  # get principal node & order cells
  print(paste0('Setting principal node on ', toupper(cell_type)))
  print(paste0('Ordering cells ...... '))

  cds[[cell_type]] <- order_cells(cds[[cell_type]], root_pr_nodes = as.vector(principal_node[[cell_type]]))

  # plot psedutime
  p <- plot_cells(
    cds = cds[[cell_type]],
    color_cells_by = "pseudotime",
    show_trajectory_graph = TRUE,
    label_roots = TRUE,
    label_leaves = FALSE,
    label_branch_points = FALSE,
  ) + umap_theme()

  pdf(paste0(fig_dir, cell_type, '_umap_pseudotime.pdf'),  width=7, height=5, useDingbats=FALSE)
  print(p)
  dev.off()

  # # add pseudotime to seurat object:
  seurat_obj[[cell_type]]$pseudotime <- pseudotime(cds[[cell_type]])

  seurat_obj[[cell_type]]$UMAP1 <- seurat_obj[[cell_type]]@reductions$umap@cell.embeddings[,1]
  seurat_obj[[cell_type]]$UMAP2 <- seurat_obj[[cell_type]]@reductions$umap@cell.embeddings[,2]

  p1 <- seurat_obj[[cell_type]]@meta.data %>%
    ggplot(aes(x=UMAP1, y=UMAP2, color=pseudotime)) +
    ggrastr::rasterise(geom_point(size=1), dpi=500, scale=0.75) +
    coord_equal() +
    scale_color_gradientn(colors=plasma(256), na.value='grey') +
    umap_theme()

  p2 <- seurat_obj[[cell_type]]@meta.data %>%
    ggplot(aes(x=UMAP1, y=UMAP2, color=pathology.group)) +
    ggrastr::rasterise(geom_point(size=1), dpi=500, scale=0.75) +
    coord_equal() +
    #scale_color_gradientn(colors=plasma(256), na.value='grey') +
    umap_theme()    

  # assemble with patchwork
  pdf(paste0(fig_dir, 'pathology_', cell_type, '_umap_pseudotime.pdf'),  width=8, height=4, useDingbats=FALSE)
  print(p1 + p2)
  dev.off()
}

[1] "Setting principal node on OPC"
[1] "Ordering cells ...... "


Cells aren't colored in a way that allows them to be grouped.


In [13]:
# convert nested list of Seurat object into Rpy2 object 
seurat_obj =  robjects.ListVector(
                        {
                            cell_type: seurat_obj[cell_type]
                     
                            for cell_type in seurat_obj.keys()
                        }
                    )

## **Module dynamics**

Here we use the hdWGCNA function `PlotModuleTrajectory` to visualize how the module eigengenes change throughout the pseudotime trajectories for each co-expression module. 

This function requires the name of the column in the Seurat object’s meta data where the pseudotime information is stored.



In [14]:
%%R -i seurat_obj -o seurat_obj

conflicts_prefer(dplyr::select)

fig_dir = paste0("../results/hdWGCNA/Pseudotime/ModuleDynamics/", save_prefix, '/')
if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

for (cell_type in names(seurat_obj)){

  print(paste0('Plotting module dynamics for ', toupper(cell_type)))


  # # Save plot to PDF
  # p  <- PlotModuleTrajectory(
  #   seurat_obj[[cell_type]],
  #   pseudotime_col = 'pseudotime'
  # )

  # pdf(paste0(fig_dir, cell_type, '_module_dynamics.pdf'),  width=4, height=4, useDingbats=FALSE)
  # print(p)
  # dev.off()

  # ME scores along the pseuodotime trajectory


  # cut pseudotime into bins using
  seurat_obj[[cell_type]] <- BinPseudotime(seurat_obj[[cell_type]], n_bins=c(10, 50, 100))

  # add MEs to seurat metadata
  MEs <- GetMEs(seurat_obj[[cell_type]], harmonized=TRUE)
  modules <- GetModules(seurat_obj[[cell_type]])
  mods <- levels(modules$module)
  mods <- mods[mods!='grey']
  module_colors <- modules %>% dplyr::select(c(module, color)) %>% distinct()

  rownames(module_colors) <- module_colors$module
  mod_colors <- module_colors[mods, 'color']
  seurat_obj[[cell_type]]@meta.data <- cbind(seurat_obj[[cell_type]]@meta.data, MEs)

  # compute the average ME score in each bin:
  features <- mods


  # Retrieve metadata of the specified cell type from the Seurat object
  meta_data <- seurat_obj[[cell_type]]@meta.data
  # Repair duplicate column names using make.names()
  names(meta_data) <- make.unique(names(meta_data))
  seurat_obj[[cell_type]]@meta.data <- meta_data 


  avg_scores <- seurat_obj[[cell_type]]@meta.data %>%
    group_by(pseudotime_bins_50) %>%
    select(all_of(features)) %>%
    summarise_all(mean)

  colnames(avg_scores)[1] <- 'bin'

  plot_df <- reshape2::melt(avg_scores)

  p1 <- ggplot(plot_df, aes(x = as.numeric(bin), y = value, color=variable)) +
    geom_point() +
    geom_smooth() +
    scale_color_manual(values=mod_colors)+
    xlab('Pseudotime') +
    ylab('Module Eigengene') +
    theme(
      axis.ticks.x = element_blank(),
      axis.text.x = element_blank()
    ) +
    NoLegend()

  plot_list <- ModuleFeaturePlot(seurat_obj[[cell_type]], order=TRUE, raster=TRUE, raster_dpi=400, alpha=0.7, title=FALSE)

  patch <- wrap_plots(plot_list, ncol=1)

  pdf(paste0(fig_dir, cell_type, '_module_dynamics.pdf'), width=6, height=6)
  print(p1 + facet_wrap(~variable, ncol=3, scales='free')) | patch
  dev.off()

}

[conflicted] Will prefer dplyr::select over any other package.
[1] "Plotting module dynamics for OPC"
Adding missing grouping variables: `pseudotime_bins_50`
[1] "OPC-M1"
[1] "OPC-M2"
[1] "OPC-M3"
[1] "OPC-M4"
[1] "OPC-M5"
[1] "OPC-M6"
[1] "OPC-M7"
[1] "OPC-M8"
`geom_smooth()` using method = 'loess' and formula = 'y ~ x'


Using bin as id variables
In addition: Warning message:
In min(xx[xx > upper]) : no non-missing arguments to min; returning Inf


In [15]:
# convert nested list of Seurat object into Rpy2 object 
seurat_obj =  robjects.ListVector(
                        {
                            cell_type: seurat_obj[cell_type]
                     
                            for cell_type in seurat_obj.keys()
                        }
                    )

Based on these dynamics, we can see which co-expression modules are turning their expression programs on or off throughout the transition from stem cells to mature erythrocytes.

We can also compare module dynamics for multiple trajectories simultaneously by passing more than one pseudotime_col parameters to PlotModuleTrajectory.

### **Proportion of diagnosis in each pseudotime bin**


In [16]:
%%R -i seurat_obj -o seurat_obj



fig_dir = paste0("../results/hdWGCNA/Pseudotime/ModuleDynamics/", save_prefix, '/')
if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

color_scheme_celltype <- list()
color_scheme_celltype[['Excitatory']] = 'aquamarine3'
color_scheme_celltype[['Inhibitory']] = 'darkolivegreen3'
color_scheme_celltype[['Astrocyte']] = 'brown3'
color_scheme_celltype[['Oligodendrocyte']] = 'darkorchid3'
color_scheme_celltype[['Microglia']] = 'deepskyblue3'
color_scheme_celltype[['OPC']] = 'deeppink3'



for (cell_type in celltypes) {
  tryCatch({
    bins <- levels(seurat_obj[[cell_type]]@meta.data$pseudotime_bins_50)
    proportion_df <- data.frame()

    for (i in 1:length(bins)) {
      bin <- bins[i]

      cur_meta <- subset(seurat_obj[[cell_type]]@meta.data, pseudotime_bins_50 == bin)
      cur_meta$pathology.group <- ifelse(cur_meta$pathology.group != 'no', 'AD', cur_meta$pathology.group)

      cur_df <- data.frame(table(cur_meta$pathology.group) / nrow(cur_meta))
      cur_df <- cur_df %>% dplyr::rename(c(Diagnosis = Var1, proportion = Freq))
      cur_df$bin_num <- i
      cur_df$bin <- bin
      proportion_df <- rbind(proportion_df, cur_df)
    }

    proportion_cor <- cor.test(
      subset(proportion_df, Diagnosis == 'AD') %>% .$bin_num,
      subset(proportion_df, Diagnosis == 'AD') %>% .$proportion,
      method = 'pearson'
    )

    p <- ggplot(proportion_df, aes(x = bin_num, y = proportion, fill = Diagnosis, color = Diagnosis)) +
      ylab('scaled feature') +
      geom_point() +
      ylim(0.5, 1)

    p <- p +
      geom_smooth(se = FALSE) +
      scale_color_manual(values = c('gray', 'seagreen'))

    p <- p +
      theme(
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank()
      )

    pdf(paste0(fig_dir, cell_type, '_diagnosis_proportion.pdf'), width = 5, height = 2)
    print(p)
    dev.off()
  }, error = function(e) {
    message("Error processing celltype: ", cell_type)
  })
}



`geom_smooth()` using method = 'loess' and formula = 'y ~ x'


In addition: Warning messages:
1: Removed 50 rows containing non-finite values (`stat_smooth()`). 
2: Removed 50 rows containing missing values (`geom_point()`). 
3: Removed 8 rows containing missing values (`geom_smooth()`). 


In [17]:
# convert nested list of Seurat object into Rpy2 object 
seurat_obj =  robjects.ListVector(
                        {
                            cell_type: seurat_obj[cell_type]
                     
                            for cell_type in seurat_obj.keys()
                        }
                    )

# **Save Seurat Object**

In [18]:
%%R -i seurat_obj -o seurat_obj

dat_dir = paste0("../results/hdWGCNA/SeuratObject/", save_prefix, '/')

if (!dir.exists(dat_dir)) {
  dir.create(dat_dir, recursive=TRUE)
}

for (cell_type in names(seurat_obj)){
    
    print(paste0('Saving hdWGCNA object in hdWGCNA Experiment for ', toupper(cell_type)))

    saveRDS(seurat_obj[[cell_type]], file=paste0(dat_dir, cell_type, '_hdWGCNA_object.rds'))
}

[1] "Saving hdWGCNA object in hdWGCNA Experiment for OPC"
